In [1]:
import os
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import pickle
from glob import glob
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import random_split
import pandas as pd
import time
from einops import rearrange,reduce,repeat
from tqdm import trange

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
class MyLSTM(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers):
        super(MyLSTM, self).__init__()

        # Defining some parameters
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        #Defining the layers
        # RNN Layer
        self.lstm = nn.LSTM(input_size, hidden_dim, n_layers, batch_first=True)   
        # Fully connected layer
        self.fc = nn.Linear(hidden_dim, output_size)
    
    def forward(self, x):
        
        batch_size = x.size(0)

        #Initializing hidden state for first input using method defined below
        h_t, c_t = self.init_hidden(batch_size)

        # Passing in the input and hidden state into the model and obtaining outputs
        out, (h_t, c_t) = self.lstm(x, (h_t, c_t))
        
        # Reshaping the outputs such that it can be fit into the fully connected layer
        #out = out.contiguous().view(-1, self.hidden_dim)
        out = self.fc(out)
        
        return out, (h_t, c_t)
    
    def init_hidden(self, batch_size):
        # This method generates the first hidden state of zeros which we'll use in the forward pass
        h_0 = torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(device)
        c_0 =  torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(device)       
         # We'll send the tensor holding the hidden state to the device we specified earlier as well
        return h_0, c_0

In [18]:
class PVDataset(torch.utils.data.Dataset):
    def __init__(self, 
                 x_data_path,
                 y_data_path):
        super(PVDataset, self).__init__()
        
        self.X = np.load(x_data_path)
        self.y = np.load(y_data_path)

    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        sample_x = self.X[idx]
        sample_y = self.y[idx]
        return torch.from_numpy(sample_x).float(), torch.from_numpy(sample_y.flatten()).float()

In [41]:
batch_size = 1024
in_dim = 4
out_dim = 30*2
hidden_dim = 60
num_layers = 3
learning_rate = 0.01
decay_rate = 0.95
num_epoch = 1000
name = 'rnn_2_60_layer3'
model_name = name + '_model.pth'
submission_name = name + '_submission.csv'

In [42]:
model = MyLSTM(input_size=in_dim, output_size=out_dim, hidden_dim=hidden_dim, n_layers=num_layers).to(device)   #maximum number of hidden size is 120

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=decay_rate)  # stepwise learning rate decay
loss_fun = nn.MSELoss()

In [43]:
def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

def train_epoch(train_loader, model, optimizer, loss_function):
    model.train()
    train_mse = []
    for inp, tgt in train_loader:
        inp = inp.to(device)
        tgt = tgt.to(device)
       
        # print(inp.shape,tgt.shape)
        pred,_ = model(inp)
        pred = pred[:,-1, :].squeeze(1)
        loss = loss_function(pred, tgt)
        train_mse.append(loss.item()) 
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    train_mse = round(np.sqrt(np.mean(train_mse)),5)
    return train_mse

def eval_epoch(valid_loader, model, loss_function):
    model.eval()
    valid_mse = []
    with torch.no_grad():
        for inp, tgt in valid_loader:
            inp = inp.to(device)
            tgt = tgt.to(device)
            
            loss = 0
            pred,_ = model(inp)
            #print(tgt.shape,pred.shape)
            pred = pred[:,-1, :].squeeze(1)
            loss = loss_function(pred, tgt)
            valid_mse.append(loss.item())
            
        valid_mse = round(np.sqrt(np.mean(valid_mse)), 5)
    return valid_mse

In [44]:
train_rmse = []
valid_rmse = []
min_rmse = 10e8

dataset = PVDataset('train_X.npy', 'train_y.npy')

for epoch in range(num_epoch):
    start = time.time()
    train_set, valid_set = random_split(
        dataset=dataset,
        lengths=[0.9, 0.1],
        generator=torch.Generator().manual_seed(0)
    )

    # create dataloaders
    train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)
    valid_loader = torch.utils.data.DataLoader(valid_set, batch_size=batch_size, shuffle=False)

    model.train() # if you use dropout or batchnorm. 
    train_rmse.append(train_epoch(train_loader, model, optimizer, loss_fun))

    model.eval()
    valid_rmse.append(eval_epoch(valid_loader, model, loss_fun))

    # save the best model
    if valid_rmse[-1] < min_rmse:
        min_rmse = valid_rmse[-1] 
        best_model = model
        torch.save([best_model, epoch, get_lr(optimizer)], model_name)

    end = time.time()
    
    # Early Stopping
    if (len(train_rmse) > 100 and np.mean(valid_rmse[-5:]) >= np.mean(valid_rmse[-10:-5])):
            break       

    # Learning Rate Decay        
    scheduler.step()
    
    print("Epoch {} | T: {:0.2f} | Train RMSE: {:0.5f} | Valid RMSE: {:0.5f}".format(epoch + 1, (end-start) / 60, train_rmse[-1], valid_rmse[-1]))

Epoch 1 | T: 0.07 | Train RMSE: 0.18435 | Valid RMSE: 0.10121
Epoch 2 | T: 0.07 | Train RMSE: 0.05915 | Valid RMSE: 0.04953
Epoch 3 | T: 0.07 | Train RMSE: 0.04113 | Valid RMSE: 0.06719
Epoch 4 | T: 0.07 | Train RMSE: 0.03736 | Valid RMSE: 0.05957
Epoch 5 | T: 0.07 | Train RMSE: 0.03559 | Valid RMSE: 0.05000
Epoch 6 | T: 0.07 | Train RMSE: 0.03401 | Valid RMSE: 0.06721
Epoch 7 | T: 0.07 | Train RMSE: 0.03556 | Valid RMSE: 0.04880
Epoch 8 | T: 0.07 | Train RMSE: 0.03339 | Valid RMSE: 0.11953
Epoch 9 | T: 0.07 | Train RMSE: 0.04212 | Valid RMSE: 0.04109
Epoch 10 | T: 0.08 | Train RMSE: 0.03258 | Valid RMSE: 0.04326
Epoch 11 | T: 0.07 | Train RMSE: 0.03248 | Valid RMSE: 0.03640
Epoch 12 | T: 0.07 | Train RMSE: 0.03184 | Valid RMSE: 0.04041
Epoch 13 | T: 0.07 | Train RMSE: 0.03173 | Valid RMSE: 0.04129
Epoch 14 | T: 0.07 | Train RMSE: 0.03165 | Valid RMSE: 0.03608
Epoch 15 | T: 0.08 | Train RMSE: 0.03137 | Valid RMSE: 0.03765
Epoch 16 | T: 0.07 | Train RMSE: 0.03125 | Valid RMSE: 0.05039
E

In [45]:
best_model, _, _ = torch.load(model_name)
best_model

MyLSTM(
  (lstm): LSTM(4, 60, num_layers=3, batch_first=True)
  (fc): Linear(in_features=60, out_features=60, bias=True)
)

In [46]:
test_path = "./val_in/val_in/"
test_pkl_list = glob(os.path.join(test_path, '*'))
test_pkl_list.sort()

test_preds = []
for idx in range(len(test_pkl_list)):
    with open(test_pkl_list[idx], 'rb') as f:
        test_sample = pickle.load(f)
        # the index of agent to be predicted 
        pred_id = np.where(test_sample["track_id"] == test_sample['agent_id'])[0][0]
        # input: p_in & v_in; output: p_out
        p_in_raw = test_sample['p_in'][pred_id]
        v_in_raw = test_sample['v_in'][pred_id]
        lane_scene = test_sample['lane']
        lane_norm_scene = test_sample['lane_norm']
        lane_norm_scene_normalized = lane_norm_scene/np.linalg.norm(lane_norm_scene,axis=1,keepdims=True)

        # Normalization
        min_vecs = np.min(lane_scene, axis = 0)
        max_vecs = np.max(lane_scene, axis = 0)
        
        # Normalize by vectors
        p_in_normalized = (p_in_raw - min_vecs)/(max_vecs - min_vecs)
        v_in_norm = np.linalg.norm(v_in_raw, axis=1, keepdims=True)
        v_in_norm = np.where(v_in_norm == 0.0, 1.0, v_in_norm)
        v_in_normalized = v_in_raw / v_in_norm
        # v_out_normalized = v_out_raw / np.linalg.norm(v_out_raw, axis=1, keepdims=True)
        
        #print('shape is',p_agent.shape,v_agent.shape,p_result.shape,v_result.shape)
        inp = np.concatenate((p_in_normalized,v_in_normalized),axis=1)
        inp = torch.from_numpy(inp).float().to(device).unsqueeze(0)
        preds,_ = model(inp)
        preds = preds.squeeze(0)
        pred = preds[-1, :].cpu().data.numpy()
        pred = rearrange(pred, "(b c) -> b c", c =2 )
        # De-Normalization ! 
        pred = pred * (max_vecs[:2] - min_vecs[:2]) +  min_vecs[:2]
        test_preds.append(pred)

In [47]:
# # Submission Files
sample_sub = pd.read_csv('sample_submission.csv')

# Convert to int
predictions = np.concatenate(test_preds).reshape(len(test_preds), -1)
sub_df = pd.DataFrame(np.c_[sample_sub["ID"], predictions], columns=[np.r_[["ID"], ["v" + str(i) for i in range(1, 61)]]])
sub_df['ID'] = sub_df['ID'].astype('int')
sub_df.to_csv(submission_name, index=None)
sub_df.dtypes

ID       int32
v1     float64
v2     float64
v3     float64
v4     float64
        ...   
v56    float64
v57    float64
v58    float64
v59    float64
v60    float64
Length: 61, dtype: object